# YOLOX 依存パッケージインストール(YOLOX Dependent Package Install)

In [ ]:
!git clone https://github.com/Megvii-BaseDetection/YOLOX -b 0.3.0

In [ ]:
%cd YOLOX

!pip install -U pip && pip install -r requirements.txt
!pip install -v -e .  

In [ ]:
!pip install loguru thop ninja onnx onnxsim onnxruntime

# PyCocoToolsインストール(PyCocoTools Install)

In [ ]:
!pip install cython
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

# データセットダウンロード(Download Dataset)

In [ ]:
%cd /content/

use_sample_image = True

if use_sample_image:
    !git clone https://github.com/Kazuhito00/YOLOX-Colaboratory-Training-Sample.git

# 学習/検証データ分割(Train/Validation split data)

In [6]:
import os

# 独自のデータを使用する場合は、パスを指定してください
# Please fill in the path if you want to use your own data
if use_sample_image:
    dataset_directory = 'YOLOX-Colaboratory-Training-Sample/02.annotation_data'
else:
    dataset_directory = ''

# 学習/検証データパス(train/validation data path)
train_directory = './train'
validation_directory = './validation'

# 学習データ格納ディレクトリ作成(Create training data storage directory)
os.makedirs(train_directory, exist_ok=True)
# 検証データ格納ディレクトリ作成(Create verification data storage directory)
os.makedirs(validation_directory, exist_ok=True)

In [7]:
import glob
import shutil
import random

# 学習データの割合(Percentage of training data)
train_ratio = 0.8

# コピー元ファイルリスト取得(Get copy source file list)
annotation_list = sorted(glob.glob(dataset_directory + '/*.xml'))
image_list = sorted(glob.glob(dataset_directory + '/*.jpg'))

file_num = len(annotation_list)

# インデックスシャッフル(shuffle)
index_list = list(range(file_num - 1))
random.shuffle(index_list)

for count, index in enumerate(index_list):
    if count < int(file_num * train_ratio):
        # 学習用データ(Training Data)
        shutil.copy2(annotation_list[index], train_directory)
        shutil.copy2(image_list[index], train_directory)
    else:
        # 検証用データ(Validation Data)
        shutil.copy2(annotation_list[index], validation_directory)
        shutil.copy2(image_list[index], validation_directory)

# Pascal VOC形式 を MS COCO形式へ変換(Convert Pascal VOC format to MS COCO format)

In [ ]:
!git clone https://github.com/Kazuhito00/convert_voc_to_coco.git

In [ ]:
!python convert_voc_to_coco/convert_voc_to_coco.py \
    train train/train_annotations.json \
    --start_image_id=0
!python convert_voc_to_coco/convert_voc_to_coco.py \
    validation validation/validation_annotations.json \
    --start_image_id=10000000

# 学習データディレクトリ準備(Training data directory preparation)

In [ ]:
!mkdir dataset
!mkdir dataset/images
!mkdir dataset/images/train2017
!mkdir dataset/images/val2017
!mkdir dataset/annotations

!cp -rf train/*.jpg dataset/images/train2017
!cp -rf validation/*.jpg dataset/images/val2017
!cp -rf train/train_annotations.json dataset/annotations
!cp -rf validation/validation_annotations.json dataset/annotations

# コンフィグコピー
<!--
![image](https://user-images.githubusercontent.com/37477845/135283504-254ea817-345e-4665-828a-4c6034645ed1.png)
-->


In [11]:
if use_sample_image:
    !cp /content/YOLOX-Colaboratory-Training-Sample/03.config/nano.py /content/YOLOX

# モデル訓練

In [ ]:
%cd /content/YOLOX/
!wget https://github.com/Megvii-BaseDetection/storage/releases/download/0.0.1/yolox_nano.pth

In [ ]:
!cp tools/train.py ./

In [ ]:
!python train.py \
    -f nano.py \
    -d 1 \
    -b 16 \
    --fp16 \
    -o \
    -c yolox_nano.pth

# 推論テスト(Inference test)

In [ ]:
!cp tools/demo.py ./

In [ ]:
TEST_IMAGE_PATH = "/content/YOLOX-Colaboratory-Training-Sample/01.image/000050.jpg"
MODEL_PATH = "/content/YOLOX/YOLOX_outputs/nano/best_ckpt.pth"

!python demo.py image \
    -f nano.py \
    -c {MODEL_PATH} \
    --path {TEST_IMAGE_PATH} \
    --conf 0.25 \
    --nms 0.45 \
    --tsize 640 \
    --save_result \
    --device gpu

In [17]:
# from PIL import Image

# OUTPUT_IMAGE_PATH = "/content/YOLOX/YOLOX_outputs/nano/vis_res/2021_09_29_17_46_56/000050.jpg" 
# Image.open(OUTPUT_IMAGE_PATH)

# ONNX出力(Export ONNX Model)

In [ ]:
!cp tools/export_onnx.py ./

In [ ]:
!python export_onnx.py \
    --output-name yolox_nano.onnx \
    -n yolox-nano \
    -f nano.py \
    -c {MODEL_PATH}

In [ ]:
!cp demo/ONNXRuntime/onnx_inference.py ./

In [ ]:
!python onnx_inference.py \
    -m yolox_nano.onnx \
    -i {TEST_IMAGE_PATH} \
    -o ./ \
    -s 0.3 \
    --input_shape 416,416

In [ ]:
from PIL import Image

OUTPUT_IMAGE_PATH = "000050.jpg" 
Image.open(OUTPUT_IMAGE_PATH)

# ONNX -> TensorFlow 変換

In [ ]:
!pip install onnx-tf

In [ ]:
!onnx-tf convert \
    -i yolox_nano.onnx \
    -o yolox_nano_pb

# TensorFlow -> TensorFlow-Lite 変換

In [ ]:
!pip install tf-nightly

In [26]:
import tensorflow as tf

In [ ]:
%cd /content/YOLOX

In [ ]:
# ダイナミックレンジ量子化
converter = tf.lite.TFLiteConverter.from_saved_model('yolox_nano_pb')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open('yolox_nano_dynamic_range_quantize.tflite', 'wb').write(tflite_quantized_model)

In [ ]:
# 半精度浮動小数点数の量子化
converter = tf.lite.TFLiteConverter.from_saved_model('yolox_nano_pb')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_quantized_model = converter.convert()

open('yolox_nano_float16_quantize.tflite', 'wb').write(tflite_quantized_model)

In [ ]:
# 完全整数量子化
import glob
import numpy as np
from PIL import Image

test_image_pathlist = glob.glob('/content/YOLOX-Colaboratory-Training-Sample/01.image/*.jpg')
test_image_pathlist = test_image_pathlist[:100]
print(len(test_image_pathlist))

def representative_dataset():
    for test_image_path in test_image_pathlist:
        image = np.array(Image.open(test_image_path))
        image = image.astype('float32')
        image = tf.image.resize(image, (416, 416))
        image = image - 127.5
        image = image * 0.007843
        image = tf.transpose(image, perm=[2, 0, 1])
        image = tf.reshape(image, [1, 3, 416, 416])
        yield [image]

converter = tf.lite.TFLiteConverter.from_saved_model('yolox_nano_pb')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
tflite_quantized_model = converter.convert()

open('yolox_nano_int8_quantize.tflite', 'wb').write(tflite_quantized_model)

In [ ]:
# 完全整数量子化(入力含む)
import glob
import numpy as np
from PIL import Image

test_image_pathlist = glob.glob('/content/YOLOX-Colaboratory-Training-Sample/01.image/*.jpg')
test_image_pathlist = test_image_pathlist[:100]
print(len(test_image_pathlist))

def representative_dataset():
    for test_image_path in test_image_pathlist:
        image = np.array(Image.open(test_image_path))
        image = image.astype('float32')
        image = tf.image.resize(image, (416, 416))
        image = image - 127.5
        image = image * 0.007843
        image = tf.transpose(image, perm=[2, 0, 1])
        image = tf.reshape(image, [1, 3, 416, 416])
        yield [image]

converter = tf.lite.TFLiteConverter.from_saved_model('yolox_nano_pb')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
tflite_quantized_model = converter.convert()

open('yolox_nano_only_int8_quantize.tflite', 'wb').write(tflite_quantized_model)